# Welcome To The Galapagos Setup

The Galapagos Middleware presents an orchestration framework to create multi-FPGA and CPU networks and can be built on top of any device that has a Galapagos Hypervisor.

<a id='own_example'></a>

# Setup

## Setup Environment Variables

The following cells follow the initial setup. The first sets up the environment variables needed by the scripts below.

In [1]:
%%bash

galapagos_path=/home/user/galapagos
cd $galapagos_path

vivado_version=2018.1
hls_version=2018.1
galapagos_board=adm-8k5-debug
vivado_path=/opt/Xilinx/Vivado
hls_path=/opt/Xilinx/Vivado

source init.sh $galapagos_path $vivado_path $hls_path $vivado_version $hls_version
galapagos-update-board $galapagos_board

bash: fg: %%bash: no such job
Updating galapagos initialization...


## Make Middleware IP


Next we need to build the IP cores used by the middleware layer to connect different Galapagos objects

In [2]:
%%bash
source $GALAPAGOS_VIVADO_PATH/settings64.sh

cd $GALAPAGOS_PATH
make hlsmiddleware  

bash: fg: %%bash: no such job
mkdir -p /home/user/galapagos/hlsBuild
make -C /home/user/galapagos/middleware hlsmiddleware
make[1]: Entering directory '/home/user/galapagos/middleware'
make -C /home/user/galapagos/middleware/hls
make[2]: Entering directory '/home/user/galapagos/middleware/hls'
for dir in galapagos_bridge/. router/. network_bridge_tcp/. network_bridge_raw/. network_bridge_ethernet/.; do \
	make -C $dir -f Makefile all; \
done
make[3]: Entering directory '/home/user/galapagos/middleware/hls/galapagos_bridge'
mkdir -p /home/user/galapagos/hlsBuild/adm-8k5-debug/ip/galapagos_bridge
vivado_hls /home/user/galapagos/middleware/hls/galapagos_bridge/generate_hls.tcl

****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2018.1 (64-bit)
  **** SW Build 2188600 on Wed Apr  4 18:39:19 MDT 2018
  **** IP Build 2185939 on Wed Apr  4 20:55:05 MDT 2018
    ** Copyright 1986-2018 Xilinx, Inc. All Rights Reserved.

source /opt/Xilinx/Vivado/2018.1/scripts/vivado_hls/hls.

INFO: [BIND 205-101] Binding ...
INFO: [BIND 205-100] Finished micro-architecture generation.
INFO: [HLS 200-111]  Elapsed time: 0.04 seconds; current allocated memory: 118.325 MB.
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-10] -- Generating RTL for module 'g2N_ingress'
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [RTGEN 206-100] Finished creating RTL model for 'g2N_ingress'.
INFO: [HLS 200-111]  Elapsed time: 0.04 seconds; current allocated memory: 118.551 MB.
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-10] -- Generating RTL for module 'g2N_egress'
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [RTGEN 206-100] Finished creating RTL model for 'g2N_egress'.
INFO: [HLS 200-111]  Elapsed time: 0.07 seconds; current allocated memory: 119.398 MB.
INFO: [HLS 200-10] ------------------------------

 #define SSDM_OP_ATTR __attribute__ ((nothrow)) __attribute__((overloadable))
                                                                            ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:166:36: note: in expansion of macro 'SSDM_OP_ATTR'
     _uint1_ _ssdm_op_IfNbRead(...) SSDM_OP_ATTR;
                                    ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:153:76: warning: 'bitwidth' attribute directive ignored [-Wattributes]
 #define SSDM_OP_ATTR __attribute__ ((nothrow)) __attribute__((overloadable))
                                                                            ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:166:36: note: in expansion of macro 'SSDM_OP_ATTR'
     _uint1_ _ssdm_op_IfNbRead(...) SSDM_OP_ATTR;
                                    ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:153:76: warning: 'overloadable' attribute directive ignored [-Wattributes]
 #define SSDM_OP_ATTR __attribut

     _uint1_  _ssdm_StreamNbWrite(...) SSDM_OP_ATTR;
                                       ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:153:76: warning: 'overloadable' attribute directive ignored [-Wattributes]
 #define SSDM_OP_ATTR __attribute__ ((nothrow)) __attribute__((overloadable))
                                                                            ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:176:39: note: in expansion of macro 'SSDM_OP_ATTR'
     _uint1_  _ssdm_StreamCanRead(...) SSDM_OP_ATTR;
                                       ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:153:76: warning: 'bitwidth' attribute directive ignored [-Wattributes]
 #define SSDM_OP_ATTR __attribute__ ((nothrow)) __attribute__((overloadable))
                                                                            ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:176:39: note: in expansion of macro 'SSDM_OP_ATTR'
     _uint1_  _ssdm_

 #define SSDM_SPEC_ATTR __attribute__ ((nothrow)) __attribute__((overloadable))
                                                                              ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:195:33: note: in expansion of macro 'SSDM_SPEC_ATTR'
     void _ssdm_op_SpecPort(...) SSDM_SPEC_ATTR;
                                 ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:152:78: warning: 'overloadable' attribute directive ignored [-Wattributes]
 #define SSDM_SPEC_ATTR __attribute__ ((nothrow)) __attribute__((overloadable))
                                                                              ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:196:39: note: in expansion of macro 'SSDM_SPEC_ATTR'
     void _ssdm_op_SpecConnection(...) SSDM_SPEC_ATTR;
                                       ^
/opt/Xilinx/Vivado/2018.1/include/etc/autopilot_ssdm_op.h:152:78: warning: 'overloadable' attribute directive ignored [-Wattributes]
 #define SSD

...

INFO: [IMPL 213-200] Port 'rxGalapagosBridge_V.data' is mapped to 'TDATA' by default.
INFO: [IMPL 213-200] Port 'rxGalapagosBridge_V.keep' is mapped to 'TKEEP' by default.
INFO: [IMPL 213-200] Port 'rxGalapagosBridge_V.last' is mapped to 'TLAST' by default.
INFO: [IMPL 213-200] Port 'rxData_V.data' is mapped to 'TDATA' by default.
INFO: [IMPL 213-200] Port 'rxData_V.keep' is mapped to 'TKEEP' by default.
INFO: [IMPL 213-200] Port 'rxData_V.last' is mapped to 'TLAST' by default.
INFO: [IMPL 213-200] Port 'rxMetaData_V_V' is mapped to 'TDATA' by default.
INFO: [IMPL 213-200] Port 'txStatus_V_V' is mapped to 'TDATA' by default.
INFO: [HLS 200-111] Finished generating all RTL models Time (s): cpu = 00:01:23 ; elapsed = 00:00:50 . Memory (MB): peak = 749.832 ; gain = 401.266 ; free physical = 30721 ; free virtual = 100396
INFO: [SYSC 207-301] Generating SystemC RTL for network_bridge_tcp.
INFO: [VHDL 208-304] Generating VHDL RTL for network_bridge_tcp.
INFO: [VLOG 209-307] Generating Verilo

INFO: [XFORM 203-602] Inlining function 'reverseEndian64_data' into 'app_to_raw' (/home/user/galapagos/middleware/hls/network_bridge_raw/src/raw_bridge.cpp:112) automatically.
INFO: [HLS 200-111] Finished Checking Synthesizability Time (s): cpu = 00:00:51 ; elapsed = 00:00:21 . Memory (MB): peak = 476.957 ; gain = 128.395 ; free physical = 31023 ; free virtual = 100723
INFO: [XFORM 203-1101] Packing variable 'to_app.V' (/home/user/galapagos/middleware/hls/network_bridge_raw/src/raw_bridge.cpp:154) into a 89-bit variable.
INFO: [XFORM 203-1101] Packing variable 'from_raw.V' (/home/user/galapagos/middleware/hls/network_bridge_raw/src/raw_bridge.cpp:155) into a 73-bit variable.
INFO: [XFORM 203-1101] Packing variable 'from_app.V' (/home/user/galapagos/middleware/hls/network_bridge_raw/src/raw_bridge.cpp:156) into a 89-bit variable.
INFO: [XFORM 203-1101] Packing variable 'to_raw.V' (/home/user/galapagos/middleware/hls/network_bridge_raw/src/raw_bridge.cpp:157) into a 73-bit variable.
INFO

INFO: [IMPL 213-200] Port 'from_app_V.data' is mapped to 'TDATA' by default.
INFO: [IMPL 213-200] Port 'from_app_V.dest' is mapped to 'TDEST' by default.
INFO: [IMPL 213-200] Port 'from_app_V.last' is mapped to 'TLAST' by default.
INFO: [IMPL 213-200] Port 'from_app_V.id' is mapped to 'TID' by default.
INFO: [IMPL 213-200] Port 'from_app_V.keep' is mapped to 'TKEEP' by default.
INFO: [IMPL 213-200] Port 'from_raw_V.data' is mapped to 'TDATA' by default.
INFO: [IMPL 213-200] Port 'from_raw_V.last' is mapped to 'TLAST' by default.
INFO: [IMPL 213-200] Port 'from_raw_V.tkeep' is mapped to 'TKEEP' by default.
INFO: [IMPL 213-200] Port 'to_app_V.data' is mapped to 'TDATA' by default.
INFO: [IMPL 213-200] Port 'to_app_V.dest' is mapped to 'TDEST' by default.
INFO: [IMPL 213-200] Port 'to_app_V.last' is mapped to 'TLAST' by default.
INFO: [IMPL 213-200] Port 'to_app_V.id' is mapped to 'TID' by default.
INFO: [IMPL 213-200] Port 'to_app_V.keep' is mapped to 'TKEEP' by default.
INFO: [IMPL 213-

INFO: [SCHED 204-11] Starting scheduling ...
INFO: [SCHED 204-61] Pipelining function 'eth_to_app'.
INFO: [SCHED 204-61] Pipelining result : Target II = 1, Final II = 1, Depth = 2.
	fifo read on port 'mac_address_V' (1.75 ns)
	'icmp' operation ('tmp_i', /home/user/galapagos/middleware/hls/network_bridge_ethernet/src/ethernet_bridge.cpp:92) (1.39 ns)
	'select' operation ('storemerge1_cast_i_c', /home/user/galapagos/middleware/hls/network_bridge_ethernet/src/ethernet_bridge.cpp:92) (0.19 ns)
	'store' operation (/home/user/galapagos/middleware/hls/network_bridge_ethernet/src/ethernet_bridge.cpp:93) of variable 'storemerge1_cast_i_c', /home/user/galapagos/middleware/hls/network_bridge_ethernet/src/ethernet_bridge.cpp:92 on static variable 'state_V_1' (1 ns)
INFO: [SCHED 204-11] Finished scheduling.
INFO: [HLS 200-111]  Elapsed time: 0.07 seconds; current allocated memory: 121.847 MB.
INFO: [BIND 205-100] Starting micro-architecture generation ...
INFO: [BIND 205-101] Performing variable li

INFO: [IMPL 213-200] Port 'from_eth_V.data' is mapped to 'TDATA' by default.
INFO: [IMPL 213-200] Port 'from_eth_V.last' is mapped to 'TLAST' by default.
INFO: [IMPL 213-200] Port 'from_eth_V.tkeep' is mapped to 'TKEEP' by default.
INFO: [IMPL 213-200] Port 'to_app_V.data' is mapped to 'TDATA' by default.
INFO: [IMPL 213-200] Port 'to_app_V.last' is mapped to 'TLAST' by default.
INFO: [IMPL 213-200] Port 'to_app_V.tdest' is mapped to 'TDEST' by default.
INFO: [IMPL 213-200] Port 'to_app_V.tkeep' is mapped to 'TKEEP' by default.
INFO: [IMPL 213-200] Port 'to_eth_V.data' is mapped to 'TDATA' by default.
INFO: [IMPL 213-200] Port 'to_eth_V.last' is mapped to 'TLAST' by default.
INFO: [IMPL 213-200] Port 'to_eth_V.tkeep' is mapped to 'TKEEP' by default.

****** Vivado v2018.1 (64-bit)
  **** SW Build 2188600 on Wed Apr  4 18:39:19 MDT 2018
  **** IP Build 2185939 on Wed Apr  4 20:55:05 MDT 2018
    ** Copyright 1986-2018 Xilinx, Inc. All Rights Reserved.

source run_ippack.tcl -notrace
INF

## Creating User Kernels 

To create a user kernel it is very similar to that of creating the hlsmiddleware in the previous section.
In the following we have an example.

First we set the board we wish to target:

In [3]:
%%bash

galapagos-update-board adm-8k5-debug

bash: fg: %%bash: no such job
Updating galapagos initialization...


Next we make the kernel:

In [4]:
%%bash 

cd /home/user/HCAL_HLS4ML
make hls

bash: fg: %%bash: no such job
vivado_hls generate_hls.tcl

****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2018.1 (64-bit)
  **** SW Build 2188600 on Wed Apr  4 18:39:19 MDT 2018
  **** IP Build 2185939 on Wed Apr  4 20:55:05 MDT 2018
    ** Copyright 1986-2018 Xilinx, Inc. All Rights Reserved.

source /opt/Xilinx/Vivado/2018.1/scripts/vivado_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/opt/Xilinx/Vivado/2018.1/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'user' on host '7830a829c43a' (Linux_x86_64 version 4.4.0-83-generic) on Mon Sep 09 03:09:38 +0000 2019
INFO: [HLS 200-10] In directory '/home/user/HCAL_HLS4ML'
INFO: [HLS 200-10] Creating and opening project '/home/user/galapagos/hlsBuild/adm-8k5-debug/ip/hls4ml_hcal'.
INFO: [HLS 200-10] Creating and opening solution '/home/user/galapagos/hlsBuild/adm-8k5-debug/ip/hls4ml_hcal/solution1'.
INFO: [HLS 200-10] Setting target device to 'xcku115-flva1517-2-e'
INFO: [HLS 200-10] Adding design file

INFO: [XFORM 203-501] Unrolling loop 'ResetAccum' (/home/user/HCAL_HLS4ML/nnet_utils/nnet_layer.h:101) in function 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely.
INFO: [XFORM 203-501] Unrolling loop 'Accum1' (/home/user/HCAL_HLS4ML/nnet_utils/nnet_layer.h:109) in function 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely.
INFO: [XFORM 203-501] Unrolling loop 'Accum2' (/home/user/HCAL_HLS4ML/nnet_utils/nnet_layer.h:113) in function 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' completely.
INFO: [XFORM 203-501] Unrolling loop 'Result' (/home/user/HCAL_HLS4ML/nnet_utils/nnet_layer.h:119) in function 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' 

INFO: [XFORM 203-602] Inlining function '__hls_fptosi_double_i32' into 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config1>' (/home/user/HCAL_HLS4ML/nnet_utils/nnet_layer.h:73) automatically.
INFO: [XFORM 203-602] Inlining function '__hls_fptosi_double_i32' into 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config2>' (/home/user/HCAL_HLS4ML/nnet_utils/nnet_layer.h:73) automatically.
INFO: [XFORM 203-602] Inlining function '__hls_fptosi_double_i32' into 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config3>' (/home/user/HCAL_HLS4ML/nnet_utils/nnet_layer.h:73) automatically.
INFO: [XFORM 203-622] Instantiating function 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config3>'(/home/user/HCAL_HLS4ML/nn

INFO: [XFORM 203-622] Instantiating function 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config1>.0.0.0.0.0.0.0.0.0'(/home/user/HCAL_HLS4ML/nnet_utils/nnet_layer.h:62:28) to 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config1>.0.0.0.0.0.0.0.0.0.0' at call site (/home/user/HCAL_HLS4ML/ereg_v1.cpp:64) by setting 'biases[8].V' to 'b1.8'.
INFO: [XFORM 203-622] Instantiating function 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config1>.0.0.0.0.0.0.0.0.0.0'(/home/user/HCAL_HLS4ML/nnet_utils/nnet_layer.h:62:28) to 'nnet::compute_layer<ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, ap_fixed<32, 14, (ap_q_mode)5, (ap_o_mode)3, 0>, config1>.0.0.0.0.0.0.0.0.0.0.0' at call site (/home/user/HCAL_HLS4ML/ereg_v1.cpp:64) by setting 'biases[9].V' to 'b1.9'.
INFO: [XFORM 203-622] 

INFO: [HLS 200-111] Finished Architecture Synthesis Time (s): cpu = 00:01:34 ; elapsed = 00:00:51 . Memory (MB): peak = 684.566 ; gain = 336.004 ; free physical = 30543 ; free virtual = 100414
INFO: [HLS 200-10] Starting hardware synthesis ...
INFO: [HLS 200-10] Synthesizing 'hls4ml_hcal' ...
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-42] -- Implementing module 'compute_layer_0_0_0_1'
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [SCHED 204-11] Starting scheduling ...
INFO: [SCHED 204-61] Pipelining function 'compute_layer.0.0.0..1'.
INFO: [SCHED 204-61] Pipelining result : Target II = 1, Final II = 1, Depth = 5.
INFO: [SCHED 204-11] Finished scheduling.
INFO: [HLS 200-111]  Elapsed time: 53.68 seconds; current allocated memory: 379.358 MB.
INFO: [BIND 205-100] Starting micro-architecture generation ...
INFO: [BIND 205-101] Performing variable lifetime analysis.
INFO: [BIND 205-101] Expl

INFO: [SCHED 204-11] Finished scheduling.
INFO: [HLS 200-111]  Elapsed time: 1.04 seconds; current allocated memory: 393.605 MB.
INFO: [BIND 205-100] Starting micro-architecture generation ...
INFO: [BIND 205-101] Performing variable lifetime analysis.
INFO: [BIND 205-101] Exploring resource sharing.
INFO: [BIND 205-101] Binding ...
INFO: [BIND 205-100] Finished micro-architecture generation.
INFO: [HLS 200-111]  Elapsed time: 0.38 seconds; current allocated memory: 394.312 MB.
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [HLS 200-10] -- Generating RTL for module 'compute_layer_0_0_0_1'
INFO: [HLS 200-10] ----------------------------------------------------------------
INFO: [SYN 201-210] Renamed object name 'hls4ml_hcal_mul_32s_12ns_44_3_0' to 'hls4ml_hcal_mul_3bkb' due to the length limit 20
INFO: [SYN 201-210] Renamed object name 'hls4ml_hcal_mul_32s_15s_46_3_0' to 'hls4ml_hcal_mul_3cud' due to the length limit 20
INFO: [SYN 201-210] Rena

INFO: [RTGEN 206-100] Generating core module 'hls4ml_hcal_mul_3Bew': 1 instance(s).
INFO: [RTGEN 206-100] Generating core module 'hls4ml_hcal_mul_3CeG': 5 instance(s).
INFO: [RTGEN 206-100] Generating core module 'hls4ml_hcal_mul_3Ffa': 1 instance(s).
INFO: [RTGEN 206-100] Generating core module 'hls4ml_hcal_mul_3Gfk': 1 instance(s).
INFO: [RTGEN 206-100] Generating core module 'hls4ml_hcal_mul_3bkb': 13 instance(s).
INFO: [RTGEN 206-100] Generating core module 'hls4ml_hcal_mul_3cud': 2 instance(s).
INFO: [RTGEN 206-100] Generating core module 'hls4ml_hcal_mul_3dEe': 5 instance(s).
INFO: [RTGEN 206-100] Generating core module 'hls4ml_hcal_mul_3eOg': 3 instance(s).
INFO: [RTGEN 206-100] Generating core module 'hls4ml_hcal_mul_3fYi': 5 instance(s).
INFO: [RTGEN 206-100] Generating core module 'hls4ml_hcal_mul_3g8j': 6 instance(s).
INFO: [RTGEN 206-100] Generating core module 'hls4ml_hcal_mul_3ibs': 7 instance(s).
INFO: [RTGEN 206-100] Generating core module 'hls4ml_hcal_mul_3jbC': 2 inst

INFO: [RTMG 210-282] Generating pipelined core: 'hls4ml_hcal_mul_3Ffa_MulnS_30'
INFO: [RTMG 210-282] Generating pipelined core: 'hls4ml_hcal_mul_3Gfk_MulnS_31'
INFO: [RTMG 210-282] Generating pipelined core: 'hls4ml_hcal_mul_3Hfu_MulnS_32'
INFO: [HLS 200-111] Finished generating all RTL models Time (s): cpu = 00:01:53 ; elapsed = 00:01:14 . Memory (MB): peak = 812.566 ; gain = 464.004 ; free physical = 30452 ; free virtual = 100350
INFO: [SYSC 207-301] Generating SystemC RTL for hls4ml_hcal.
INFO: [VHDL 208-304] Generating VHDL RTL for hls4ml_hcal.
INFO: [VLOG 209-307] Generating Verilog RTL for hls4ml_hcal.
INFO: [IMPL 213-8] Exporting RTL as a Vivado IP.

****** Vivado v2018.1 (64-bit)
  **** SW Build 2188600 on Wed Apr  4 18:39:19 MDT 2018
  **** IP Build 2185939 on Wed Apr  4 20:55:05 MDT 2018
    ** Copyright 1986-2018 Xilinx, Inc. All Rights Reserved.

source run_ippack.tcl -notrace
INFO: [IP_Flow 19-234] Refreshing IP repositories
INFO: [IP_Flow 19-1704] No user IP repositories 

This will place the generated IP cores in `<galapagos_home>/hlsBuild/<board_name>/ip`. Here we can see all the IP cores that we have created for this board. This includes the middleware IP cores and the User IP cores

In [5]:
%%bash

cd $GALAPAGOS_PATH/hlsBuild/$GALAPAGOS_BOARD_NAME/ip
ls

bash: fg: %%bash: no such job
ethernet_bridge   network_bridge_ethernet  raw_bridge
galapagos_bridge  network_bridge_raw       width32
hls4ml_hcal       network_bridge_tcp       width48


Now that we have created our IP cores, we can describe them in a cluster and stitch them together. <a href='http://127.0.0.1:9000/notebooks/02_middleware.ipynb'>Using the Middleware'</a>